## 7 Optimised LightGBM + Conformal Quantile Regression (CQR)

### 7 .1 Motivation  
Our baseline LightGBM‐CQR (Section 6) already beat the linear QR in pinball loss but:

* **Hyper-parameters were default / heuristic.**  
* No GPU use (training time ≈ 45 s per fold on CPU).  
* Categorical handling relied on one-hots → wider matrices, slower.

The goal here is to **auto-tune** the model for each target quantile  
τ ∈ {0.05, 0.25, 0.50, 0.75, 0.95} using Optuna, while preserving conformal validity.

---

### 7 .2 Data pre-processing (unchanged)

| Step | Action | Why |
|------|--------|-----|
| 1 | `feat_cols` = 29 predictors selected in Stage 5 | dimension already pruned |
| 2 | Numeric columns unchanged; categorical columns (`token`, `day_of_week`, `momentum_bucket`, `vol_regime`, `trend_regime`) cast to `pd.Categorical` | lets LightGBM apply *optimal split for categories* |
| 3 | Forward- then back-fill within each token → no NaNs | LightGBM supports NaNs but conformal scores prefer finite data |

---

### 7 .3 Search space design   <sub>(from LGB docs + practice)</sub>

| Group | Hyper-parameter | Range / Prior | Rationale |
|-------|-----------------|---------------|-----------|
| **Tree complexity** | `num_leaves` | log-uniform [32, 256] | controls model capacity <br>keep ≤ 2⁸ to avoid leaf sparsity |
| | `max_depth` | int [4, 14] | prevents overly deep trees |
| | `min_data_in_leaf` | log-uniform [5, 300] | regularises in presence of class imbalance |
| **Learning** | `learning_rate` | log-uniform [0.005, 0.1] | lower lr ↔ more trees ↔ better generalisation |
| **Bagging / feature frac** | `feature_fraction` | uniform [0.4, 1.0] | column subsampling |
| | `bagging_fraction` | uniform [0.4, 1.0] | row subsampling |
| | `bagging_freq` | int [0, 15] | how often to re-sample |
| **ℓ¹ / ℓ² penalties** | `lambda_l1`, `lambda_l2` | log-uniform (1e-4, 5] | combats over-fitting |
| **Node split score** | `min_gain_to_split` | uniform [0, 0.4] | extra regularisation |

*Search strategy*: **TPE sampler** (Optuna) + **Hyperband pruner**  
→ quickly drops poor configs after ~200 trees, explores promising ones up to 8000.

---

### 7 .4 Conformal wrapper (same as baseline)

1. Split **train / calibration / test** inside each trial (70 / 15 / 15 %).  
2. Fit LightGBM on *train*.  
3. Compute residuals on *calibration* set.  
4. For τ < 0.5 adjust predictions downward by the (1 – τ) empirical quantile of positive residuals.  
   For τ > 0.5 adjust upward (negative residuals).  
5. Evaluate **pinball loss** on held-out *validation* fold.  
   → returned to Optuna.

This yields **finite-sample, distribution-free coverage** (Lei et al., 2018).

---

### 7 .5 Hardware notes  

* **CPU build** (OpenMP) is retained – GPU not compulsory.  
  `device_type` switches automatically if a GPU-capable wheel is later installed.  
* `n_jobs = -1` uses all 24 logical threads (32 GB RAM easily handles 8 k-tree boosters).  
* `study.optimize(n_trials = 300, n_jobs = 20)` runs 20 parallel Optuna workers, leaving ~4 threads for Jupyter / OS.

---

### 7 .6 Stopping & logging  

* **Early stopping**: after 400 rounds without improvement on *validation*.  
  (Early stopping uses LightGBM’s internal 5 % quantile metric.)  
* **Pruner**: Hyperband halts unpromising trials early → ~60 % time-saving.  
* **Persistence**: best params per τ are stored in `best_lgb_cqr_params.json` for reproducible Stage 8 re-fits.

---

### References  

* Ke et al. (2017) *LightGBM: A Highly Efficient Gradient Boosting Decision Tree.*  
* Lei, Romano & Barbieri (2018) *Conformal Prediction Under Covariate Shift.*  
* Optuna docs: <https://optuna.org/>  
* LightGBM tuning guide: <https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html>



In [26]:
# ============================================================
# 0.  Imports & environment check
# ============================================================
import os, gc, json, joblib, warnings, datetime as dt
import numpy as np
import pandas as pd

import lightgbm as lgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
from optuna.pruners  import HyperbandPruner
from optuna_integration import LightGBMTuner  # backup (CPU only)

warnings.filterwarnings("ignore", category=UserWarning)

print("LightGBM:", lgb.__version__, "| Optuna:", optuna.__version__)

# -------- Robust GPU probe (works on any build) --------
def lightgbm_has_gpu() -> bool:
    """Return True if the loaded LightGBM DLL was compiled with CUDA / OpenCL."""
    try:
        # available since v3.3.0; returns 'CPU' on cpu-only builds
        return lgb.get_device_name(0) != "CPU"
    except AttributeError:         # very old 3.2.x or earlier
        return False

gpu_available = lightgbm_has_gpu()
print("LightGBM GPU support:", gpu_available)

LightGBM: 3.3.5 | Optuna: 3.6.0
LightGBM GPU support: False


In [27]:
# ============================================================
# 1.  Load data + minimal preprocessing
#    (uses your features_v1_tail.csv)
# ============================================================

TARGET   = "return_72h"

cat_cols = ["day_of_week","momentum_bucket", "extreme_flag1", "tail_asym","vol_regime", "token"]
num_cols = [c for c in df.columns
            if c not in cat_cols + ["timestamp", TARGET]]

FEATURES = cat_cols + num_cols     # preserve order

# LightGBM prefers pandas with category dtype for cat feats
X_df = df[FEATURES].copy()
for c in cat_cols:
    X_df[c] = X_df[c].astype("category")

y = df[TARGET].values
X = X_df                     # ★ keep as DataFrame, NOT .values
cat_idx = [X.columns.get_loc(c) for c in cat_cols]   # optional
print(f"Matrix  → {X.shape[0]:,} rows × {X.shape[1]} cols")



Matrix  → 5,703 rows × 34 cols


In [28]:
# 3.  Optuna search space 
# ============================================================

def suggest_params(trial, tau: float) -> dict:
    return {
        # ----- core CQR settings -----
        "objective" : "quantile",
        "metric"    : "quantile",
        "alpha"     : tau,
        "device_type": "gpu" if gpu_available else "cpu",

        # ----- tree complexity -----
        "learning_rate" : trial.suggest_float("lr",      0.005, 0.1,  log=True),
        "num_leaves"    : trial.suggest_int(  "leaves",      32, 256, log=True),
        "max_depth"     : trial.suggest_int(  "depth",        4, 14),
        "min_data_in_leaf":
                          trial.suggest_int(  "min_leaf",     5, 300, log=True),

        # ----- randomness & regularisation -----
        "feature_fraction": trial.suggest_float("feat_frac", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bag_frac",  0.4, 1.0),
        "bagging_freq"    : trial.suggest_int(  "bag_freq",      0, 15),

        # **FIX**: low bound must be > 0 when log=True  → use 1e-8
        "lambda_l1" : trial.suggest_float("l1", 1e-8, 5.0, log=True),
        "lambda_l2" : trial.suggest_float("l2", 1e-8, 5.0, log=True),

        "min_gain_to_split":
                          trial.suggest_float("gamma",     0.0, 0.4),

        # ----- training length -----
        "num_iterations"        : 8000,
        "early_stopping_round"  : 400,      # (LightGBM’s param without the “s”)
        "verbosity"             : -1,
        "seed"                  : 42,
        "n_jobs"                : -1,       # all 24 logical threads
    }

In [29]:
# ------------------------------------------------------------
#  3.  Objective function uses the *existing* X, y, cat_idx
# ------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss      # scikit-learn ≥1.1
import lightgbm as lgb

def pinball(y_true, y_pred, tau):
    """Lightweight pinball without sklearn if preferred."""
    diff = y_true - y_pred
    return np.maximum(tau*diff, (tau-1)*diff).mean()

def objective(trial, tau):
    params = suggest_params(trial, tau)

    X_tr, X_val, y_tr, y_val = train_test_split(
        X, y, test_size=0.15, random_state=trial.number)

    # ★ pass DataFrames directly
    lgb_train = lgb.Dataset(X_tr, label=y_tr)
    lgb_val   = lgb.Dataset(X_val, label=y_val, reference=lgb_train)

    booster = lgb.train(params,
                        train_set=lgb_train,
                        valid_sets=[lgb_val],
                        verbose_eval=False)

    y_hat = booster.predict(X_val, num_iteration=booster.best_iteration)
    loss  = pinball(y_val, y_hat, tau)
    trial.set_user_attr("best_iter", booster.best_iteration)
    return loss



In [ ]:
# ============================================================
# 5.  Run Optuna – **parallel** over 20 jobs
#     Do this once per quantile (0.05 … 0.95)
# ============================================================
QUANTS = [0.05, 0.25, 0.50, 0.75, 0.95]
best_params = {}

for tau in QUANTS:
    print(f"\n=== Optimising τ = {tau:.2f} ===")
    sampler = TPESampler(seed=42, multivariate=True)
    pruner  = HyperbandPruner(min_resource=200, max_resource=8000)

    study = optuna.create_study(
        direction="minimize",
        sampler=sampler,
        pruner=pruner,
        study_name=f"lgb_cqr_tau{tau:.2f}",
        storage=f"sqlite:///lgb_cqr_tau{tau:.2f}.db",
        load_if_exists=True
    )

    study.optimize(lambda t: objective(t, tau),
                   n_trials=300,
                   n_jobs=20,            # 24 threads – 4 for OS / Jupyter
                   timeout=3*3600,
                   show_progress_bar=True)

    print("Best pinball:", study.best_value,
          "| best iter:", study.best_trial.user_attrs["best_iter"])

    p = study.best_params
    p.update(objective= "quantile",
             metric   = "quantile",
             alpha    = tau,
             num_iterations = study.best_trial.user_attrs["best_iter"])

    best_params[tau] = p

# persist for later stages
json.dump(best_params, open("best_lgb_cqr_params.json", "w"), indent=2)
print("\nSaved → best_lgb_cqr_params.json")

c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-06-27 15:37:44,758] Using an existing study with name 'lgb_cqr_tau0.05' instead of creating a new one.



=== Optimising τ = 0.05 ===


Best trial: 64. Best value: 0.00791401:   0%|          | 1/300 [00:06<30:47,  6.18s/it, 6.17/10800 seconds]

[I 2025-06-27 15:37:50,927] Trial 64 finished with value: 0.007914005160411412 and parameters: {'lr': 0.050495617589756556, 'leaves': 98, 'depth': 6, 'min_leaf': 82, 'feat_frac': 0.6223357648292999, 'bag_frac': 0.7055444971536873, 'bag_freq': 2, 'l1': 0.00014674455932836737, 'l2': 8.337377325136031e-08, 'gamma': 0.32171810355752933}. Best is trial 64 with value: 0.007914005160411412.


Best trial: 61. Best value: 0.00686776:   1%|          | 3/300 [00:07<08:35,  1.74s/it, 7.13/10800 seconds]

...

[I 2025-06-27 15:48:16,220] Trial 47 finished with value: 0.020668877594099758 and parameters: {'lr': 0.005652642632709717, 'leaves': 107, 'depth': 5, 'min_leaf': 212, 'feat_frac': 0.9980545731222356, 'bag_frac': 0.9078939053224729, 'bag_freq': 0, 'l1': 1.4545910371095033e-08, 'l2': 3.188015641332999e-06, 'gamma': 0.15627967659833156}. Best is trial 21 with value: 0.018170224823683103.


Best trial: 21. Best value: 0.0181702:  16%|█▋        | 49/300 [03:50<10:56,  2.61s/it, 230.75/10800 seconds]

[I 2025-06-27 15:48:18,439] Trial 43 finished with value: 0.024698565310682447 and parameters: {'lr': 0.005820846215768352, 'leaves': 38, 'depth': 5, 'min_leaf': 177, 'feat_frac': 0.8823006732530476, 'bag_frac': 0.8052938743293379, 'bag_freq': 1, 'l1': 1.4193407663183614e-07, 'l2': 1.8772860731873585e-07, 'gamma': 0.03659884016211235}. Best is trial 21 with value: 0.018170224823683103.


Best trial: 21. Best value: 0.0181702:  17%|█▋        | 50/300 [03:55<13:32,  3.25s/it, 235.48/10800 seconds]

[I 2025-06-27 15:48:23,170] Trial 45 finished with value: 0.02114842037374556 and parameters: {'lr': 0.005307374273932499, 'leaves': 40, 'depth': 5, 'min_leaf': 142, 'feat_frac': 0.958874113783255, 'bag_frac': 0.754899882247125, 'bag_freq': 1, 'l1': 9.99403026503018e-07, 'l2': 1.0031768604926736e-07, 'gamma': 0.06904852894018039}. Best is trial 21 with value: 0.018170224823683103.
